# Task 1 - Part (A) CSP Problem Specifying

### Variables:

Each position in the path from Start (S) to Target (T) is a variable:
Let the path be a sequence of positions: P0, P1, ..., Pn
Each Pi is a variable representing a grid cell (x, y)

### Domain:
Each variable Pi has a domain:
All diagonal neighbor grid positions (x ± 1, y ± 1)
Only if within grid bounds and not an obstacle

### Constraints

##### Diagonal Move Constraint:
Each step from Pi to Pi+1 must be a diagonal move:
abs(x2 - x1) == 1 and abs(y2 - y1) == 1

##### Obstacle Avoidance:
Pi ∉ Obstacles

##### No Revisit Constraint (optional for shortest path):
All positions in the path must be unique

##### Start/End Constraint:
P0 = S, Pn = T

##### Path Continuity:
Pi+1 must be reachable from Pi with a legal move

# Task 1 - Part (B) and (C) Python Program Using OR-Tools

In [12]:
from ortools.sat.python import cp_model

grid_size = 5
start = (0, 0)  # (1,1)
target = (3, 3)  # (4,4)
obstacles = {(1, 1), (0, 2)}  # (2,2), (1,3)
max_steps = 2 * grid_size

model = cp_model.CpModel()
x = [model.NewIntVar(0, grid_size - 1, f'x{i}') for i in range(max_steps)]
y = [model.NewIntVar(0, grid_size - 1, f'y{i}') for i in range(max_steps)]
used = [model.NewBoolVar(f'used{i}') for i in range(max_steps)]

model.Add(x[0] == start[0])
model.Add(y[0] == start[1])

for i in range(max_steps - 1):
    dx = model.NewIntVar(-1, 1, f'dx{i}')
    dy = model.NewIntVar(-1, 1, f'dy{i}')
    model.Add(dx == x[i + 1] - x[i])
    model.Add(dy == y[i + 1] - y[i])
    abs_dx = model.NewIntVar(0, 1, f'abs_dx{i}')
    abs_dy = model.NewIntVar(0, 1, f'abs_dy{i}')
    model.AddAbsEquality(abs_dx, dx)
    model.AddAbsEquality(abs_dy, dy)
    is_move = model.NewBoolVar(f'is_move_{i}')
    model.Add(abs_dx + abs_dy == 2).OnlyEnforceIf(is_move)
    model.Add(abs_dx + abs_dy != 2).OnlyEnforceIf(is_move.Not())
    model.Add(used[i + 1] == is_move)

for i in range(max_steps):
    for ox, oy in obstacles:
        is_obs_x = model.NewBoolVar(f'obsx_{i}_{ox}_{oy}')
        is_obs_y = model.NewBoolVar(f'obsy_{i}_{ox}_{oy}')
        model.Add(x[i] == ox).OnlyEnforceIf(is_obs_x)
        model.Add(x[i] != ox).OnlyEnforceIf(is_obs_x.Not())
        model.Add(y[i] == oy).OnlyEnforceIf(is_obs_y)
        model.Add(y[i] != oy).OnlyEnforceIf(is_obs_y.Not())
        model.AddBoolOr([is_obs_x.Not(), is_obs_y.Not()])

reached = [model.NewBoolVar(f'reached{i}') for i in range(max_steps)]
for i in range(max_steps):
    tx = model.NewBoolVar(f'tx_{i}')
    ty = model.NewBoolVar(f'ty_{i}')
    model.Add(x[i] == target[0]).OnlyEnforceIf(tx)
    model.Add(x[i] != target[0]).OnlyEnforceIf(tx.Not())
    model.Add(y[i] == target[1]).OnlyEnforceIf(ty)
    model.Add(y[i] != target[1]).OnlyEnforceIf(ty.Not())
    model.AddBoolAnd([tx, ty]).OnlyEnforceIf(reached[i])
    model.AddBoolOr([tx.Not(), ty.Not()]).OnlyEnforceIf(reached[i].Not())

model.AddBoolOr(reached)
model.Minimize(sum(used))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Shortest diagonal path from (1,1) to (4,4):")
    for i in range(max_steps):
        xi = solver.Value(x[i])
        yi = solver.Value(y[i])
        print(f"Step {i}: ({xi+1}, {yi+1})")  # convert to 1-based
        if (xi, yi) == target:
            break
else:
    print("No solution found.")


Shortest diagonal path from (1,1) to (4,4):
Step 0: (1, 1)
Step 1: (1, 1)
Step 2: (2, 1)
Step 3: (3, 1)
Step 4: (3, 2)
Step 5: (3, 3)
Step 6: (4, 3)
Step 7: (4, 3)
Step 8: (4, 4)


# Task 2 - AI System That Helps Environmental Scientists To Predict Future Land Loss.

In [13]:
from ortools.sat.python import cp_model

grid_input = [
    [0, 1, 1, 0],
    [1, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
]

rows = len(grid_input)
cols = len(grid_input[0])

model = cp_model.CpModel()
grid = [[model.NewBoolVar(f'cell_{i}_{j}') for j in range(cols)] for i in range(rows)]

for i in range(rows):
    for j in range(cols):
        model.Add(grid[i][j] == grid_input[i][j])

perimeter_contrib = []

for i in range(rows):
    for j in range(cols):
        if grid_input[i][j] == 1:
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + dx, j + dy
                if 0 <= ni < rows and 0 <= nj < cols:
                    is_water = model.NewBoolVar(f'edge_{i}_{j}_{ni}_{nj}')
                    model.Add(grid[ni][nj] == 0).OnlyEnforceIf(is_water)
                    model.Add(grid[ni][nj] != 0).OnlyEnforceIf(is_water.Not())
                    perimeter_contrib.append(is_water)
                else:
                    perimeter_contrib.append(model.NewConstant(1))

total_perimeter = model.NewIntVar(0, rows * cols * 4, 'total_perimeter')
model.Add(total_perimeter == sum(perimeter_contrib))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
    print("Perimeter of the landmass:", solver.Value(total_perimeter))
else:
    print("No solution found.")


Perimeter of the landmass: 12


# Task 3 - CSP To Solve The TSP (Traveling Salesman Problem)

In [14]:
from ortools.sat.python import cp_model
import random

n = 10
random.seed(42)
dist = [[0 if i == j else random.randint(10, 100) for j in range(n)] for i in range(n)]
for i in range(n):
    for j in range(i + 1, n):
        dist[j][i] = dist[i][j]

model = cp_model.CpModel()
tour = [model.NewIntVar(0, n - 1, f'tour_{i}') for i in range(n)]

model.AddAllDifferent(tour)
total_distance = model.NewIntVar(0, n * 1000, 'total_distance')
segment_distances = []

for i in range(n):
    a = tour[i]
    b = tour[(i + 1) % n]
    d = model.NewIntVar(0, 1000, f'dist_{i}')
    table = []
    for x in range(n):
        for y in range(n):
            table.append([x, y, dist[x][y]])
    model.AddAllowedAssignments([a, b, d], table)
    segment_distances.append(d)

model.Add(total_distance == sum(segment_distances))
model.Minimize(total_distance)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("TSP Solution Found:")
    ordered_cities = [solver.Value(tour[i]) for i in range(n)]
    ordered_cities.append(ordered_cities[0])  # Complete the cycle
    print("Tour:", ordered_cities)
    print("Total distance:", solver.Value(total_distance))
else:
    print("No solution found.")


TSP Solution Found:
Tour: [2, 3, 5, 9, 6, 4, 8, 0, 7, 1, 2]
Total distance: 196


# Task 4 - Part (A) Transporting Packages Using Autonomous Robots

### Variables:
pos_r[r][t] = (x, y): Position of robot r at time t

carrying[r][t][p] ∈ {0,1}: Robot r is carrying package p at time t

battery[r][t]: Battery level of robot r at time t

delivered[p] ∈ {0,1}: Whether package p has been delivered

time_picked[p], time_delivered[p]: Time when package p is picked and delivered

### Domains:
Positions: Cells in 6×6 grid

Carrying: Binary

Battery: Range based on robot battery capacity (e.g., 0 to 100)

Time steps: Up to T (planning horizon)

### Constraints:
Movement: Only cardinal directions (N/S/E/W), one cell per step.

Collision Avoidance: No two robots at same cell at the same time.

Capacity Constraint: Robot can only pick package if capacity allows.

Battery Constraint: Decrease with movement, recharge at charging stations.

Pickup/Drop Rules: Packages must be picked at source and dropped at target.

Delivery Goal: All packages must be delivered within time horizon.

# Task 4 - Part (B) and (C) Implementing CSP Model Using OR-Tools

In [17]:
from ortools.sat.python import cp_model

grid_size = 6
robots = 5
packages = 10
time_horizon = 20
capacity = 10
battery_max = 15

package_sources = [(0, 0), (1, 1), (0, 2), (2, 2), (3, 1), (5, 5), (4, 0), (3, 3), (1, 5), (2, 4)]
package_targets = [(5, 0), (4, 5), (3, 0), (5, 1), (2, 5), (0, 5), (1, 0), (0, 3), (3, 4), (5, 3)]
package_weights = [2, 3, 1, 4, 2, 1, 2, 1, 3, 2]
robot_capacities = [10, 10, 10, 10, 10]
charging_stations = [(0, 0), (5, 5)]

model = cp_model.CpModel()
pos_x = [[[model.NewIntVar(0, grid_size - 1, f'robot{r}_x_{t}') for t in range(time_horizon)] for r in range(robots)]]
pos_y = [[[model.NewIntVar(0, grid_size - 1, f'robot{r}_y_{t}') for t in range(time_horizon)] for r in range(robots)]]

carrying = [[[model.NewBoolVar(f'carry_r{r}_p{p}_t{t}') for t in range(time_horizon)] for p in range(packages)] for r in range(robots)]
delivered = [model.NewBoolVar(f'delivered_p{p}') for p in range(packages)]

for r in range(robots):
    for t in range(1, time_horizon):
        dx = model.NewIntVar(-1, 1, f'dx_{r}_{t}')
        dy = model.NewIntVar(-1, 1, f'dy_{r}_{t}')
        model.Add(dx == pos_x[0][r][t] - pos_x[0][r][t - 1])
        model.Add(dy == pos_y[0][r][t] - pos_y[0][r][t - 1])
        model.AddAbsEquality(model.NewIntVar(0, 1, ''), dx)
        model.AddAbsEquality(model.NewIntVar(0, 1, ''), dy)
        prod = model.NewIntVar(-1, 1, f'prod_{r}_{t}')
        model.AddMultiplicationEquality(prod, [dx, dy])
        model.Add(prod == 0)


for t in range(time_horizon):
    for r1 in range(robots):
        for r2 in range(r1 + 1, robots):
            not_same_x = model.NewBoolVar(f'not_same_x_{r1}_{r2}_{t}')
            not_same_y = model.NewBoolVar(f'not_same_y_{r1}_{r2}_{t}')
            model.Add(pos_x[0][r1][t] != pos_x[0][r2][t]).OnlyEnforceIf(not_same_x)
            model.Add(pos_x[0][r1][t] == pos_x[0][r2][t]).OnlyEnforceIf(not_same_x.Not())
            model.Add(pos_y[0][r1][t] != pos_y[0][r2][t]).OnlyEnforceIf(not_same_y)
            model.Add(pos_y[0][r1][t] == pos_y[0][r2][t]).OnlyEnforceIf(not_same_y.Not())
            model.AddBoolOr([not_same_x, not_same_y])


for p in range(packages):
    model.AddMaxEquality(delivered[p], [carrying[r][p][t] for r in range(robots) for t in range(time_horizon)])

model.Maximize(sum(delivered))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Delivery plan:")
    for p in range(packages):
        print(f"Package {p}: Delivered =", solver.Value(delivered[p]))
else:
    print("No feasible delivery plan found.")


Delivery plan:
Package 0: Delivered = 1
Package 1: Delivered = 1
Package 2: Delivered = 1
Package 3: Delivered = 1
Package 4: Delivered = 1
Package 5: Delivered = 1
Package 6: Delivered = 1
Package 7: Delivered = 1
Package 8: Delivered = 1
Package 9: Delivered = 1


# Task 5 - Part (A) Sudoku Puzzle

### Variables:
Let each cell in the Sudoku grid be a variable. For a 9x9 grid, we have 81 variables, each representing a cell in the grid.

Each variable can take a value from the set of numbers {1, 2, 3, 4, 5, 6, 7, 8, 9}, representing the possible values for each cell.

### Domains:
The domain of each variable corresponds to the set {1, 2, 3, 4, 5, 6, 7, 8, 9}.

Some cells will have pre-filled values, which limit their domain.

### Constraints:

##### Uniqueness Constraints:

Row Constraint: Each number must appear only once in each row.

Column Constraint: Each number must appear only once in each column.

Subgrid Constraint: Each number must appear only once in each 3x3 subgrid.

##### Diagonal Sum Constraint:

The sum of the numbers on the main diagonal (from top-left to bottom-right) and the secondary diagonal (from top-right to bottom-left) must be divisible by 3.

##### Adjacent Prime Numbers Constraint:

Prime numbers (2, 3, 5, 7) cannot be adjacent horizontally or vertically

# Task 5 - Part (B) and (C) Implementing CSP Model Using OR-Tools

In [22]:
from ortools.sat.python import cp_model

primes = {2, 3, 5, 7}
model = cp_model.CpModel()
grid = [[model.NewIntVar(1, 9, f'cell_{r}_{c}') for c in range(9)] for r in range(9)]

for r in range(9):
    model.AddAllDifferent(grid[r])

for c in range(9):
    model.AddAllDifferent([grid[r][c] for r in range(9)])

for i in range(3):
    for j in range(3):
        model.AddAllDifferent([grid[i*3 + x][j*3 + y] for x in range(3) for y in range(3)])

main_diagonal = [grid[i][i] for i in range(9)]
secondary_diagonal = [grid[i][8-i] for i in range(9)]
model.AddModuloEquality(sum(main_diagonal), 0, 3)
model.AddModuloEquality(sum(secondary_diagonal), 0, 3)

for r in range(9):
    for c in range(9):
        if c < 8:
            model.Add(grid[r][c] != grid[r][c+1])
            for prime in primes:
                b1 = model.NewBoolVar(f'is_{prime}_at_{r}_{c}')
                b2 = model.NewBoolVar(f'is_{prime}_at_{r}_{c+1}')
                model.Add(grid[r][c] == prime).OnlyEnforceIf(b1)
                model.Add(grid[r][c] != prime).OnlyEnforceIf(b1.Not())
                model.Add(grid[r][c+1] == prime).OnlyEnforceIf(b2)
                model.Add(grid[r][c+1] != prime).OnlyEnforceIf(b2.Not())
                model.AddBoolOr([b1.Not(), b2.Not()])

        if r < 8:
            model.Add(grid[r][c] != grid[r+1][c])
            for prime in primes:
                b1 = model.NewBoolVar(f'is_{prime}_at_{r}_{c}')
                b2 = model.NewBoolVar(f'is_{prime}_at_{r+1}_{c}')
                model.Add(grid[r][c] == prime).OnlyEnforceIf(b1)
                model.Add(grid[r][c] != prime).OnlyEnforceIf(b1.Not())
                model.Add(grid[r+1][c] == prime).OnlyEnforceIf(b2)
                model.Add(grid[r+1][c] != prime).OnlyEnforceIf(b2.Not())
                model.AddBoolOr([b1.Not(), b2.Not()])

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10.0
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(9):
        row = [solver.Value(grid[r][c]) for c in range(9)]
        print(row)
else:
    print("No solution found")


No solution found
